# TITANIC SURVIVAL PREDICTION | NAIVE BAYES

### Importing basic Libraries

In [1]:
import pandas as pd
import numpy as np

### Importing the dataset

In [2]:
dataset= pd.read_csv('Data/train.csv')

### Exploring the dataset

In [3]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
dataset.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
dataset.shape

(891, 12)

In [6]:
dataset.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [7]:
dataset.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Data Cleaning

#### Dropping unnecessary features

In [8]:
dropped_col= ["Name", "Ticket", "Cabin", "Embarked"]
dataset.drop(columns=dropped_col, axis=1, inplace= True)
dataset.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500


In [9]:
obj_dataset = dataset.select_dtypes(include=['object']).copy()
obj_dataset.head()

,Sex
0,male
1,female
2,female
3,female
4,male


In [10]:
obj_dataset.isna().sum()

Sex    0
dtype: int64

In [11]:
pd.get_dummies(obj_dataset, drop_first= True).head()

,Sex_male
0,1
1,0
2,0
3,0
4,1


In [12]:
dataset["Sex_male"]= pd.get_dummies(obj_dataset, drop_first= True)

In [13]:
dataset.drop(columns="Sex", inplace= True)

In [14]:
dataset.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,1,0,3,22.0,1,0,7.2500,1
1,2,1,1,38.0,1,0,71.2833,0
2,3,1,3,26.0,0,0,7.9250,0
3,4,1,1,35.0,1,0,53.1000,0
4,5,0,3,35.0,0,0,8.0500,1


In [15]:
dataset.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Sex_male         uint8
dtype: object

#### Define Imputer

In [16]:
from sklearn.impute import KNNImputer
imputer= KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

#### Fit on the dataset and transform the dataset

In [17]:
imputer.fit(dataset)

KNNImputer()

In [18]:
dataset_trans= imputer.transform(dataset)

In [19]:
dataset_trans= pd.DataFrame(dataset_trans)

In [20]:
columns= {0: "PassengerId", 1: "Survived", 2: "Pclass", 3: "Age", 4: "SibSp", 5: "Parch", 6: "Fare", 7: "Sex_male" }
dataset_trans= dataset_trans.rename(columns=columns)

In [21]:
dataset_trans.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,1.0,0.0,3.0,22.0,1.0,0.0,7.2500,1.0
1,2.0,1.0,1.0,38.0,1.0,0.0,71.2833,0.0
2,3.0,1.0,3.0,26.0,0.0,0.0,7.9250,0.0
3,4.0,1.0,1.0,35.0,1.0,0.0,53.1000,0.0
4,5.0,0.0,3.0,35.0,0.0,0.0,8.0500,1.0


In [22]:
dataset_trans.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_male       0
dtype: int64

### Segregate the dataset

In [23]:
X= dataset_trans.drop("Survived", axis= 1)
X.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,1.0,3.0,22.0,1.0,0.0,7.2500,1.0
1,2.0,1.0,38.0,1.0,0.0,71.2833,0.0
2,3.0,3.0,26.0,0.0,0.0,7.9250,0.0
3,4.0,1.0,35.0,1.0,0.0,53.1000,0.0
4,5.0,3.0,35.0,0.0,0.0,8.0500,1.0


In [24]:
Y= dataset_trans.Survived
Y.head()

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Survived, dtype: float64

### Splitting the dataset into train test Set

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, Y, test_size= 0.25, random_state= 41)
print("X_train shape: {})". format(X_train.shape))
print("X_test shape: {})". format(X_test.shape))
print("y_train shape: {})". format(y_train.shape))
print("y_test shape: {})". format(y_test.shape))

X_train shape: (668, 7))
X_test shape: (223, 7))
y_train shape: (668,))
y_test shape: (223,))


### Training

In [26]:
from sklearn.naive_bayes import GaussianNB
model= GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [27]:
X_train.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_male
668,669.0,3.0,43.0,0.0,0.0,8.05,1.0
143,144.0,3.0,19.0,0.0,0.0,6.75,1.0
604,605.0,1.0,35.0,0.0,0.0,26.55,1.0
298,299.0,1.0,31.8,0.0,0.0,30.50,1.0
559,560.0,3.0,36.0,1.0,0.0,17.40,0.0


In [28]:
pass_id= float(input("Enter PassengerID: "))
pclass= float(input("Enter Pclass: "))
age= float(input("Enter Age: "))
sibsp = float(input("Enter Sibsp: "))
parch= float(input("Enter Parch: "))
fare= float(input("Enter Fare: "))
male= float(input("Enter 1 for male and 0 for female: "))
person= [[pass_id, pclass, age, sibsp, parch, fare, male]]
result= model.predict(person)
print(result)

if result==1:
    print("The person survived")
else:
    print("Person did not survived")

Enter PassengerID: 900
Enter Pclass: 3
Enter Age: 18
Enter Sibsp: 0
Enter Parch: 0
Enter Fare: 2657
Enter 1 for male and 0 for female: 0
[1.]
The person survived


c:\users\rajat verma\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


### Prediction for all test dataset

In [29]:
y_pred= model.predict(X_test)
print(np.column_stack((y_pred, y_test)))

[[0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [0. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [1. 0.]
 [0. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 

### Checking the accuracy of the model

In [30]:
from sklearn.metrics import accuracy_score
print("Accuracy of the model : {0}%".format(accuracy_score(y_test, y_pred)*100))

Accuracy of the model : 81.16591928251121%
